In [22]:
# Install libraries
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT
import matplotlib.pyplot as plt
import numpy as np
import pennylane as qml

from math import pi



In [23]:
mixed_device = qml.device("default.mixed", wires=8)


@qml.qnode(device=mixed_device)
def circuit():
    qml.QFT(wires=range(8))
    return qml.density_matrix(wires=range(8))


def get_temperatures(positions_history, graph):
    """
    Calculate the temperature of each ion based on its positions history and the graph.

    Args:
        positions_history (list): A list of positions of the ions.
        graph (nx.Graph): The graph representing the circuit.

    Returns:
        list: A list of temperatures for each ion.
    """
    temperature = [[0.0] * len(positions_history[0])]
    for i, pos in enumerate(positions_history):
        temp = temperature[-1].copy()
        for j, p in enumerate(pos):
            if i > 0:
                if p == positions_history[i - 1][j]:
                    if graph.nodes[p]["type"] == "idle":
                        temp[j] += 0.01
                    else:
                        temp[j] += 0.02
                else:
                    temp[j] += 0.03
        temperature.append(temp)
    temperature.pop(0)
    return temperature


# Create noisy circuit

from pennylane.operation import Channel

eps = eps = 1e-14


class DepolarizingChannel(Channel):
    num_params = 1
    num_wires = 2
    grad_method = "A"
    grad_recipe = ([[1, 0, 1], [-1, 0, 0]],)

    def __init__(self, p, wires, id=None):
        super().__init__(p, wires=wires, id=id)

    @staticmethod
    def compute_kraus_matrices(p):  # pylint:disable=arguments-differ
        r"""Kraus matrices representing the depolarizing channel."""
        if not 0.0 <= p <= 1.0:
            raise ValueError("p must be in the interval [0,1]")

        I = np.array([[1, 0], [0, 1]], dtype=complex)
        X = np.array([[0, 1], [1, 0]], dtype=complex)
        Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
        Z = np.array([[1, 0], [0, -1]], dtype=complex)

        pauli_2qubit = [
            np.kron(I, I),
            np.kron(X, I),
            np.kron(Y, I),
            np.kron(Z, I),
            np.kron(I, X),
            np.kron(I, Y),
            np.kron(I, Z),
            np.kron(X, X),
            np.kron(X, Y),
            np.kron(X, Z),
            np.kron(Y, X),
            np.kron(Y, Y),
            np.kron(Y, Z),
            np.kron(Z, X),
            np.kron(Z, Y),
            np.kron(Z, Z),
        ]
        return [
            np.sqrt(1 - p + eps) * pauli_2qubit[0],
            np.sqrt(p / 15 + eps) * pauli_2qubit[1],
            np.sqrt(p / 15 + eps) * pauli_2qubit[2],
            np.sqrt(p / 15 + eps) * pauli_2qubit[3],
            np.sqrt(p / 15 + eps) * pauli_2qubit[4],
            np.sqrt(p / 15 + eps) * pauli_2qubit[5],
            np.sqrt(p / 15 + eps) * pauli_2qubit[6],
            np.sqrt(p / 15 + eps) * pauli_2qubit[7],
            np.sqrt(p / 15 + eps) * pauli_2qubit[8],
            np.sqrt(p / 15 + eps) * pauli_2qubit[9],
            np.sqrt(p / 15 + eps) * pauli_2qubit[10],
            np.sqrt(p / 15 + eps) * pauli_2qubit[11],
            np.sqrt(p / 15 + eps) * pauli_2qubit[12],
            np.sqrt(p / 15 + eps) * pauli_2qubit[13],
            np.sqrt(p / 15 + eps) * pauli_2qubit[14],
            np.sqrt(p / 15 + eps) * pauli_2qubit[15],
        ]


def compiled_circuit_noisy(gates_schedule, temperature) -> qml.QNode:
    """
    Build a noisy circuit from the list of gates and the ion temperatures.

    Args:
        gates_schedule (list): A list of gates where each gate is represented as a tuple.

    Returns:
        qml.QNode: A Pennylane QNode representing the circuit.
    """
    dev = qml.device("default.mixed", wires=8)

    @qml.qnode(dev)
    def circuit():
        for i, step in enumerate(gates_schedule):
            temp = temperature[i]
            for gate in step:
                if gate[0] == "RX":
                    qml.RX(gate[1], wires=gate[2])
                elif gate[0] == "RY":
                    qml.RY(gate[1], wires=gate[2])
                elif gate[0] == "MS":
                    temp1 = temp[gate[2][0]]
                    temp2 = temp[gate[2][1]]
                    average_temp = (temp1 + temp2) / 2
                    prob = (
                        (np.pi**2 * (0.05) ** 4)
                        / 4
                        * average_temp
                        * (2 * average_temp + 1)
                    )
                    assert 0.0 <= prob <= 1.0, (
                        f"Average temperature too high: ion {gate[2][0]}: {temp1}, ion {gate[2][1]}: {temp2}"
                    )
                    qml.IsingXX(gate[1], wires=gate[2])
                    DepolarizingChannel(prob, wires=gate[2])
        return qml.density_matrix(wires=range(8))

    return circuit


def fidelity(positions_history, gates_schedule, graph) -> float:
    """
    Fidelity between the ideal and noisy circuit.

    Args:
        positions_history (list): A list of positions of the ions.
        gates_schedule (list): A list of gates where each gate is represented as a tuple.

    Returns:
        float: The fidelity of the circuit.
    """
    expected_result = circuit()
    noisy_user_result = compiled_circuit_noisy(
        gates_schedule, get_temperatures(positions_history, graph)
    )()
    noisy_user_fidelity = qml.math.fidelity(expected_result, noisy_user_result)
    print("Fidelity of the circuit when including noise:", noisy_user_fidelity)
    return noisy_user_fidelity

import networkx as nx


def create_trap_graph() -> nx.Graph:
    """Create a graph representing the Penning trap.

    The Penning trap is represented as a grid of nodes, where each node can be
    either an interaction node or a standard node. The interaction nodes are
    connected to their corresponding idle nodes, and the standard nodes are
    connected to their neighboring standard nodes.
    """

    trap = nx.Graph()

    rows = 5
    cols = 7

    interaction_nodes = [(1, 1), (1, 3), (3, 1), (3, 3), (1, 5), (3, 5)]

    for r in range(rows):
        for c in range(cols):
            base_node_id = (r, c)

            if base_node_id in interaction_nodes:
                trap.add_node(base_node_id, type="interaction")
            else:
                trap.add_node(base_node_id, type="standard")
                rest_node_id = (r, c, "idle")
                trap.add_node(rest_node_id, type="idle")
                trap.add_edge(base_node_id, rest_node_id)

    for r in range(rows):
        for c in range(cols):
            node_id = (r, c)
            if c + 1 < cols:
                neighbor_id = (r, c + 1)
                trap.add_edge(node_id, neighbor_id)
            if r + 1 < rows:
                neighbor_id = (r + 1, c)
                trap.add_edge(node_id, neighbor_id)
    return trap


import pennylane as qml
import numpy as np

mixed_device = qml.device("default.mixed", wires=8)


@qml.qnode(device=mixed_device)
def circuit():
    qml.QFT(wires=range(8))
    return qml.density_matrix(wires=range(8))


def compiled_circuit(gates_schedule) -> qml.QNode:
    """
    Build the compiled circuit from the gates schedule.

    Args:
        gates_schedule (list): A list of gates where each gate is represented as a tuple.

    Returns:
        qml.QNode: A Pennylane QNode representing the circuit.
    """

    @qml.qnode(device=mixed_device)
    def circuit():
        for step in gates_schedule:
            for gate in step:
                if gate[0] == "RX":
                    qml.RX(gate[1], wires=gate[2])
                elif gate[0] == "RY":
                    qml.RY(gate[1], wires=gate[2])
                elif gate[0] == "MS":
                    qml.IsingXX(gate[1], wires=gate[2])
        return qml.density_matrix(wires=range(8))

    return circuit


def verifier(positions_history, gates_schedule, graph) -> None:
    """
    Verify the positions and gates schedule of the circuit.

    Args:
        positions_history (list): A list of positions for each step in the circuit.
        gates_schedule (list): A list of gates where each gate is represented as a tuple.
        graph (networkx.Graph): The graph representing the Penning trap.
    """
    print("Verifying the positions history and gates schedule...")
    if len(positions_history) != len(gates_schedule):
        raise ValueError(
            f"Length of positions history ({len(positions_history)}) does not match length of gates schedule ({len(gates_schedule)})."
        )
    for i, positions in enumerate(positions_history):
        if len(positions) != 8:
            raise ValueError(f"Invalid number of ions at step {i}: {len(positions)}")
        for j, p in enumerate(positions):
            if p not in graph.nodes():
                raise ValueError(
                    f"Invalid position: {p} at step {i} for ion {j} is not part of the graph."
                )

        if i > 0:
            prev_pos_tuple = positions_history[i - 1]
            curr_pos_tuple = positions

            for ion_idx in range(len(curr_pos_tuple)):
                p_prev = prev_pos_tuple[ion_idx]
                p_curr = curr_pos_tuple[ion_idx]

                if p_prev != p_curr:
                    if not graph.has_edge(p_prev, p_curr):
                        raise ValueError(
                            f"Error: Invalid move for ion {ion_idx} from {p_prev} to {p_curr} at step {i}. Nodes are not adjacent in the graph."
                        )

            # Check for ions swapping over the same edge
            num_ions = len(curr_pos_tuple)
            for ion1_idx in range(num_ions):
                for ion2_idx in range(
                    ion1_idx + 1, num_ions
                ):  # Iterate over unique pairs
                    prev_pos_ion1 = prev_pos_tuple[ion1_idx]
                    curr_pos_ion1 = curr_pos_tuple[ion1_idx]

                    prev_pos_ion2 = prev_pos_tuple[ion2_idx]
                    curr_pos_ion2 = curr_pos_tuple[ion2_idx]

                    # Check if ion1 moved to ion2's previous spot AND ion2 moved to ion1's previous spot
                    if (
                        prev_pos_ion1 == curr_pos_ion2
                        and prev_pos_ion2 == curr_pos_ion1
                    ):
                        # Ensure they were at different positions before swapping.
                        if prev_pos_ion1 != prev_pos_ion2:
                            raise ValueError(
                                f"Error: Ions {ion1_idx} and {ion2_idx} swapped positions ({prev_pos_ion1} <-> {prev_pos_ion2}) at step {i}."
                            )

        gate = gates_schedule[i]

        for g in gate:
            # Check gate semantics
            name, param, wires = g
            if name not in ["RX", "RY", "MS"]:
                raise ValueError(
                    f"Error: Gate name at step {i} is not RX, RY, or MS. Found: {name}"
                )
            if not isinstance(name, str):
                raise ValueError(
                    f"Error: Gate name at step {i} is not a string. Found: {type(name)}"
                )
            if not isinstance(param, (float, int)):
                raise ValueError(
                    f"Error: Gate parameter at step {i} is not a float or int. Found: {type(param)}"
                )
            if not isinstance(wires, (int, list, tuple)) or (
                isinstance(wires, (list, tuple))
                and not all(isinstance(w, int) for w in wires)
            ):
                raise ValueError(
                    f"Error: Gate wires at step {i} are not an int or a list/tuple of ints. Found: {type(wires)}"
                )

            if isinstance(wires, int):
                if not (0 <= wires < 8):
                    raise ValueError(
                        f"Error: Gate wire at step {i} is out of range [0, 8). Found: {wires}"
                    )
            elif isinstance(wires, (list, tuple)):
                if not all(0 <= w < 8 for w in wires):
                    raise ValueError(
                        f"Error: One or more gate wires at step {i} are out of range [0, 8). Found: {wires}"
                    )
            if g[0] == "MS":
                ion_0 = g[2][0]
                ion_1 = g[2][1]

                pos_0 = positions[ion_0]
                pos_1 = positions[ion_1]
                if pos_0 != pos_1:
                    raise ValueError(
                        f"Error: Ions {ion_0} and {ion_1} are not at the same position {pos_0} at step {i}."
                    )
                if graph.nodes[pos_0]["type"] != "interaction":
                    raise ValueError(
                        f"Error: MS gate at step {i} is not at an interaction node. Position: {pos_0}"
                    )
                pos_next = positions_history[i + 1]
                if not (
                    positions[ion_0] == pos_next[ion_0]
                    and positions[ion_1] == pos_next[ion_1]
                ):
                    raise ValueError(
                        f"Error: Ions {ion_0} or {ion_1} moved during MS gate at step {i + 1}."
                    )
            elif g[0] == "RX" or g[0] == "RY":
                ion = g[2]
                pos_ion = positions[ion]
                if graph.nodes[pos_ion]["type"] == "interaction":
                    raise ValueError(
                        f"Error: RX/RY gate at step {i} is on interaction node. Position: {pos_ion}"
                    )
                if graph.nodes[pos_ion]["type"] == "idle":
                    raise ValueError(
                        f"Error: RX/RY gate at step {i} is on rest node. Position: {pos_ion}"
                    )
        # Check wires
        wires = [g[2] for g in gate]

        flattened_wires = []
        for wire in wires:
            if isinstance(wire, (list, tuple)):
                flattened_wires.extend(wire)
            else:
                flattened_wires.append(wire)
        if len(set(flattened_wires)) != len(flattened_wires):
            raise ValueError(
                f"Error: Duplicate wires in gate at step {i}. Wires: {flattened_wires}"
            )

        # Check for overlapping ions
        positions_set = set(positions)
        for p in positions_set:
            if p[-1] != "idle":
                corresponding_idle = (*p[:2], "idle")
                if positions.count(corresponding_idle) == 1:
                    raise ValueError(
                        f"Error: Overlapping ions at {p} with its corresponding idle position at step {i}."
                    )

        if len(positions_set) < len(positions):
            overlapping_positions = [p for p in positions if positions.count(p) > 1]

            for overlap in overlapping_positions:
                if graph.nodes[overlap]["type"] != "interaction":
                    raise ValueError(
                        f"Error: Overlapping ions at non-interaction node {overlap} at step {i}."
                    )
                if positions.count(overlap) > 2:
                    raise ValueError(
                        f"Error: More than two ions overlapping at interaction node {overlap} at step {i}."
                    )
                overlapping_ions = [
                    idx for idx, p in enumerate(positions) if p == overlap
                ]
                has_ms_gate_b = False
                has_ms_gate_d = False

                has_ms_gate = False
                for g in gates_schedule[i]:
                    if g[0] == "MS" and set(g[2]) == set(overlapping_ions):
                        has_ms_gate_d = True
                        has_ms_gate = True
                        break
                if i > 0:
                    for g in gates_schedule[i - 1]:
                        if g[0] == "MS" and set(g[2]) == set(overlapping_ions):
                            has_ms_gate_b = True
                            has_ms_gate = True
                            break
                if not has_ms_gate:
                    raise ValueError(
                        f"Error: Overlapping ions at {overlap} at step {i} without an MS gate before, or during the overlap."
                    )
                if sum([has_ms_gate_b, has_ms_gate_d]) != 1:
                    raise ValueError(
                        f"Error: Overlapping ions at {overlap} at step {i} and step {i - 1} have conflicting MS gate. Only one MS gate should be present."
                    )
    print("Positions and gates are valid.")
    print("Verifying the fidelity of the circuit without adding noise...")
    expected_result = circuit()
    user_result = compiled_circuit(gates_schedule)()
    user_fidelity = qml.math.fidelity(expected_result, user_result)
    print("Fidelity of the circuit:", user_fidelity)
    if not np.allclose(expected_result, user_result, atol=1e-5):
        raise ValueError("The compiled circuit does not implement QFT(8).")
    print("The compiled circuit implements QFT(8).")



In [ ]:
# Define allowed gate set

def Rx(theta, wires):
    """Apply RX rotation gate."""
    qml.RX(theta, wires=wires)
    
def Ry(theta, wires):
    """Apply RY rotation gate."""
    qml.RY(theta, wires=wires)
    
def MS(theta, wires):
    """Apply MS gate."""
    

def Molmer_Sorensen(t, Omega):
    ms = np.array(
        [
            [np.cos(Omega * t / 2), 0, 0, -1j * np.sin(Omega * t / 2)],
            [0, np.cos(Omega * t / 2), -1j * np.sin(Omega * t / 2), 0],
            [0, -1j * np.sin(Omega * t / 2), np.cos(Omega * t / 2), 0],
            [-1j * np.sin(Omega * t / 2), 0, 0, np.cos(Omega * t / 2)],
        ]
    )
    return ms

# Define Hadamard gate with allowed gates
def H(wires):
    """Apply Hadamard gate."""
    return Ry(pi / 2, wires=wires)*Rx(pi, wires=wires)


In [29]:
import pennylane as qml
import numpy as np

##############################
#  Low-level helper gates
##############################

def RZ_native(phi, wire):
    """Rz using only Rx/Ry."""
    qml.RX( np.pi/2, wires=wire )
    qml.RY(     phi,   wires=wire )
    qml.RX(-np.pi/2, wires=wire )

def ZZ_native(phi, wires):
    """exp(-i φ/2 Z⊗Z) via basis-rotated IsingXX (MS) gate."""
    a, b = wires
    qml.RY( np.pi/2, wires=a )
    qml.RY( np.pi/2, wires=b )
    qml.IsingXX(phi, wires=[a, b])   # ← Mølmer–Sørensen gate
    qml.RY(-np.pi/2, wires=a )
    qml.RY(-np.pi/2, wires=b )

def CP_native(phi, wires):
    """Controlled-Phase using only RZ_native + ZZ_native."""
    a, b = wires
    RZ_native( phi/2, a )
    RZ_native( phi/2, b )
    ZZ_native(-phi/2, wires)

def H_native(wire):
    """Hadamard with one Ry and one Rx."""
    qml.RY( np.pi/2, wires=wire )
    qml.RX(     np.pi, wires=wire )

##############################
#  Quantum Fourier Transform
##############################

def qft_native(wires):
    n = len(wires)
    for k in range(n):
        # 1. Hadamard on qubit k
        H_native(wires[k])
        # 2. Ladder of controlled-phase rotations
        for j in range(k+1, n):
            angle = np.pi / 2**(j - k)
            CP_native(angle, wires=[wires[j], wires[k]])
    # (Optional) bit-reversal by index swapping…

##############################
#  Example QNode
##############################

dev = qml.device("default.qubit", wires=8)

@qml.qnode(dev)
def demo_qft():
    qft_native(list(range(8)))
    return qml.state()

state = demo_qft()
print(state[:16])  # show the first few amplitudes


[-0.00115042+0.06248941j -0.00115042+0.06248941j  0.06248941+0.00115042j
  0.06248941+0.00115042j  0.04500016-0.04337322j  0.04500016-0.04337322j
 -0.04337322-0.04500016j -0.04337322-0.04500016j  0.02497651-0.05729244j
  0.02497651-0.05729244j -0.05729244-0.02497651j -0.05729244-0.02497651j
 -0.05817294+0.02285081j -0.05817294+0.02285081j  0.02285081+0.05817294j
  0.02285081+0.05817294j]


In [42]:
import pennylane as qml
import numpy as np

##############################
#  Low-level helper gates
##############################

def RZ_native(phi, wire):
    qml.RX( np.pi/2, wires=wire )
    qml.RY(     phi,   wires=wire )
    qml.RX(-np.pi/2, wires=wire )

def ZZ_native(phi, wires):
    a, b = wires
    qml.RY( np.pi/2, wires=a )
    qml.RY( np.pi/2, wires=b )
    qml.IsingXX(phi, wires=[a, b])
    qml.RY(-np.pi/2, wires=a )
    qml.RY(-np.pi/2, wires=b )

def CP_native(phi, wires):
    a, b = wires
    RZ_native( phi/2, a )
    RZ_native( phi/2, b )
    ZZ_native(-phi/2, wires)

def H_native(wire):
    qml.RY( np.pi/2, wires=wire )
    qml.RX(     np.pi, wires=wire )

##############################
#  Native QFT implementation
##############################

def qft_native(wires):
    n = len(wires)
    for k in range(n):
        H_native(wires[k])
        for j in range(k+1, n):
            angle = np.pi / 2**(j - k)
            CP_native(angle, wires=[wires[j], wires[k]])

##############################
#  Devices & QNodes
##############################

n_qubits = 8
wires = list(range(n_qubits))
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit_native():
    # start in |0…0⟩
    qft_native(wires)
    return qml.state()

@qml.qnode(dev)
def circuit_builtin():
    qml.QFT(wires=wires)
    return qml.state()

##############################
#  Test: fidelity check
##############################

state_native  = circuit_native()
state_builtin = circuit_builtin()

# Classical bit-reversal of the native output:
def classical_bit_reversal(state):
    n = int(np.log2(state.size))
    state = state.reshape([2]*n)
    # reverse the axes
    state = state.transpose(list(range(n-1, -1, -1)))
    return state.reshape(-1)

state_nat_rev = classical_bit_reversal(state_native)

fid = np.abs(np.vdot(state_nat_rev, state_builtin))**2
print(f"Fidelity after classical reversal: {fid:.12f}")
assert np.isclose(fid, 1.0, atol=1e-8), "Still not matching after classical reversal!"




Fidelity after classical reversal: 0.000000000000


AssertionError: Still not matching after classical reversal!

In [50]:
import pennylane as qml
import numpy as np

###############################################################################
#  1) Native decomposition primitives
###############################################################################

def RZ_native(phi, wire):
    """Rz(φ) via Rx/Ry Euler ZX decomposition."""
    qml.RX(  np.pi/2, wires=wire )
    qml.RY(      phi, wires=wire )
    qml.RX( -np.pi/2, wires=wire )

def CNOT_native(control, target):
    """
    CNOT ≃ RY(-π/2)ₜ • IsingXX(π/2)₍c,t₎ • RY(π/2)ₜ
    up to a global phase.
    """
    qml.RY(-np.pi/2, wires=target)
    qml.IsingXX( np.pi/2, wires=[control, target] )
    qml.RY( np.pi/2, wires=target)

def CP_via_CNOT(phi, control, target):
    """
    CP(φ) = CNOT • (I⊗Rz(φ)) • CNOT
    """
    CNOT_native(control, target)
    RZ_native(phi, target)
    CNOT_native(control, target)

def H_native(wire):
    """Hadamard via Ry(π/2) then Rx(π)."""
    qml.RY( np.pi/2, wires=wire )
    qml.RX(      np.pi, wires=wire )

def bit_reversal(wires):
    """Swap wire i with wire N-1-i."""
    n = len(wires)
    for i in range(n//2):
        qml.SWAP(wires=[wires[i], wires[n-1-i]])


###############################################################################
#  2) Eight-qubit QFT
###############################################################################

def qft_native_v2(wires):
    n = len(wires)
    # 1) Hadamard + controlled-phase ladder
    for k in range(n):
        H_native(wires[k])
        for j in range(k+1, n):
            angle = np.pi / 2**(j - k)
            CP_via_CNOT(angle, control=wires[j], target=wires[k])
    # 2) bit-reversal to match PennyLane’s qml.QFT
    bit_reversal(wires)


###############################################################################
#  3) Example QNode and fidelity test
###############################################################################

n_qubits = 3
wires    = list(range(n_qubits))
dev      = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit_native():
    qft_native_v2(wires)
    return qml.state()

@qml.qnode(dev)
def circuit_builtin():
    qml.QFT(wires=wires)
    return qml.state()

# run and compare
state_native  = circuit_native()
state_builtin = circuit_builtin()

# clean fidelity call
fid = np.abs(np.vdot(state_native, state_builtin))**2
print(f"Native-CNOT-based QFT fidelity: {fid:.12f}")
assert qml.math.isclose(fid, 1.0, atol=1e-8), "Mismatch remains!"



Native-CNOT-based QFT fidelity: 0.073223304703


AssertionError: Mismatch remains!

In [51]:
state_builtin

array([0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j,
       0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j])

In [59]:
import math
import networkx as nx
from collections import deque


def compile_qft():
    trap = create_trap_graph()

    # Standard BFS to a single goal
    def bfs_path(start, goals, occupied):
        if start in goals:
            return [start]
        visited = {start}
        queue = deque([[start]])
        while queue:
            path = queue.popleft()
            node = path[-1]
            for nei in trap.neighbors(node):
                if nei in occupied or nei in visited:
                    continue
                new_path = path + [nei]
                if nei in goals:
                    return new_path
                visited.add(nei)
                queue.append(new_path)
        return None

    # Try each target in 'goals' until one yields a path
    def find_path_to_targets(start, goals, occupied):
        for goal in goals:
            path = bfs_path(start, {goal}, occupied)
            if path is not None:
                return path
        return None

    # Find any empty standard/interaction node
    def find_any_empty(exclude):
        for n in trap.nodes():
            if len(n) == 2 and n not in exclude:
                return n
        raise RuntimeError("No empty node found")

    # Initial placement near center
    center = (2, 3)
    nbrs = [n for n in trap.neighbors(center) if len(n) == 2]
    positions = {
        0: center,
        1: nbrs[0],
        2: nbrs[1],
        3: nbrs[2],
        4: (2, 2),
        5: (1, 3),
        6: (2, 4),
        7: (3, 3),
    }

    positions_history = [[positions[q] for q in range(8)]]
    gates_schedule = []

    parking = [(0,0), (0,6), (4,0), (4,6), (0,1), (4,5), (1,6)]
    park_idx = 0

    for i in range(7):
        # 1) Move control i to center
        if positions[i] != center:
            occ = set(positions.values()) - {positions[i]}
            path_i = bfs_path(positions[i], {center}, occ)
            if path_i is None:
                raise RuntimeError(f"Cannot move control {i} to center")
            for step in path_i[1:]:
                positions[i] = step
                positions_history.append([positions[q] for q in range(8)])

        # 2) Hadamard on i = RY(pi/2); RX(pi)
        gates_schedule.append([("RY", math.pi/2, i)])
        positions_history.append(positions_history[-1].copy())
        gates_schedule.append([("RX", math.pi, i)])
        positions_history.append(positions_history[-1].copy())

        # 3) Controlled-phase for j > i
        for j in range(i+1, 8):
            start_j = positions[j]

            # a) Ensure at least one free neighbor of center
            free_nbrs = {n for n in trap.neighbors(center)
                         if len(n)==2 and n not in positions.values()}
            if not free_nbrs:
                # Evict someone
                occupied_nbrs = [q for q,p in positions.items()
                                 if p in trap.neighbors(center)
                                 and q not in (i,j)]
                k = occupied_nbrs[0]
                exclude = set(positions.values()) | set(trap.neighbors(center)) | {center}
                buffer_spot = find_any_empty(exclude)
                path_k = bfs_path(positions[k], {buffer_spot},
                                  set(positions.values()) - {positions[k]})
                for s in path_k[1:]:
                    positions[k] = s
                    positions_history.append([positions[q] for q in range(8)])
                free_nbrs = {n for n in trap.neighbors(center)
                             if len(n)==2 and n not in positions.values()}

            # b) Move j into one of those free neighbors (only if reachable)
            occ = set(positions.values()) - {positions[j], center}
            path_j = find_path_to_targets(positions[j], free_nbrs, occ)
            if path_j is None:
                raise RuntimeError(f"Cannot route qubit {j} to any free neighbor of center")
            for s in path_j[1:]:
                positions[j] = s
                positions_history.append([positions[q] for q in range(8)])

            # c) Pre-rotations
            gates_schedule.append([
                ("RY", -math.pi/2, i),
                ("RY", -math.pi/2, j),
            ])
            positions_history.append(positions_history[-1].copy())

            # d) MS gate
            theta = math.pi / (2 ** (j - i))
            gates_schedule.append([("MS", theta, (i, j))])
            positions_history.append(positions_history[-1].copy())

            # e) Post-rotations
            gates_schedule.append([
                ("RY", math.pi/2, i),
                ("RY", math.pi/2, j),
            ])
            positions_history.append(positions_history[-1].copy())

            # f) Move j back (or park)
            target = start_j
            if start_j in trap.neighbors(center):
                exclude = set(positions.values()) | set(trap.neighbors(center)) | {center}
                target = find_any_empty(exclude)
            occ = set(positions.values()) - {positions[j]}
            path_back = bfs_path(positions[j], {target}, occ)
            if path_back is None:
                raise RuntimeError(f"Cannot move qubit {j} back to {target}")
            for s in path_back[1:]:
                positions[j] = s
                positions_history.append([positions[q] for q in range(8)])

        # 4) Park control i
        park = parking[park_idx]
        park_idx += 1
        if positions[i] != park:
            occ = set(positions.values()) - {positions[i]}
            path_p = bfs_path(positions[i], {park}, occ)
            if path_p is None:
                raise RuntimeError(f"Cannot park qubit {i} at {park}")
            for s in path_p[1:]:
                positions[i] = s
                positions_history.append([positions[q] for q in range(8)])

    # 5) Final physical swaps (0<->7, 1<->6, 2<->5, 3<->4)
    for k in range(4):
        l = 7 - k
        buf = find_any_empty(set(positions.values()))
        for (src, dst) in [(k, buf), (l, positions_history[0][k]), (k, positions_history[0][l])]:
            path_s = bfs_path(positions[src], {dst}, set(positions.values()) - {positions[src]})
            if path_s is None:
                raise RuntimeError(f"Cannot swap qubit {src} to {dst}")
            for s in path_s[1:]:
                positions[src] = s
                positions_history.append([positions[q] for q in range(8)])

    return positions_history, gates_schedule

# Usage
positions_history, gates_schedule = compile_qft()

# Verify and print fidelity
verifier(positions_history, gates_schedule, create_trap_graph())
print("Fidelity:", fidelity(positions_history, gates_schedule, create_trap_graph()))


RuntimeError: Cannot route qubit 1 to any free neighbor of center

In [80]:
def cphase_ms(phi, wires, *, physical_only=False):
    """
    CPHASE(phi) realised with Rx/Ry + a single MS gate.

    Args
    ----
    phi (float): phase angle ϕ
    wires (Sequence[int]): two qubit indices (control, target in any order)
    physical_only (bool): if True, decompose the local Z-rotations into
                          explicit Rx/Ry pulses; otherwise use virtual Zs
                          (PhaseShift), which a simulator or real HW frame
                          update can handle for free.
    """
    w0, w1 = wires

    # --- local Z rotations (−ϕ/2 on each qubit) ---------------------------
    if physical_only:
        # Rz(α) = Ry(π/2) · Rx(α) · Ry(−π/2)
        for w in (w0, w1):
            qml.RY(np.pi/2, wires=w)
            qml.RX(-phi/2, wires=w)
            qml.RY(-np.pi/2, wires=w)
    else:
        qml.PhaseShift(-phi/2, wires=w0)
        qml.PhaseShift(-phi/2, wires=w1)

    # --- change XX → ZZ basis, run MS, change back -----------------------
    qml.RY(np.pi/2, wires=w0)
    qml.RY(np.pi/2, wires=w1)

    # PennyLane’s IsingXX implements  exp(-i θ/2 X⊗X)
    qml.IsingXX(-phi, wires=[w0, w1])

    qml.RY(-np.pi/2, wires=w0)
    qml.RY(-np.pi/2, wires=w1)
    
    dev = qml.device("default.qubit", wires=2)
    
    



In [85]:
dev = qml.device("default.qubit", wires=8)

# --- the template from the previous answer -------------------------------
def cphase_ms(phi, wires, physical_only=False):
    w0, w1 = wires
    if physical_only:
        for w in (w0, w1):
            qml.RY(np.pi/2, wires=w)
            qml.RX(-phi/2, wires=w)
            qml.RY(-np.pi/2, wires=w)
    else:
        qml.PhaseShift(-phi/2, wires=w0)
        qml.PhaseShift(-phi/2, wires=w1)

    qml.RY(np.pi/2, wires=w0)
    qml.RY(np.pi/2, wires=w1)
    qml.IsingXX(-phi, wires=[w0, w1])
    qml.RY(-np.pi/2, wires=w0)
    qml.RY(-np.pi/2, wires=w1)

dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def mat_template(phi):
    cphase_ms(phi, [0, 1])
    return qml.state()

@qml.qnode(dev)
def mat_builtin(phi):
    qml.ControlledPhaseShift(phi, wires=[0, 1])
    return qml.state()

phi = 0.321
print("max |difference| between unitaries:",
      np.max(np.abs(mat_template(phi) - mat_builtin(phi))))

max |difference| between unitaries: 0.16032778379163667


In [94]:
import pennylane as qml
import numpy as np

# 1) Set up an 8-qubit simulator
dev = qml.device("default.qubit", wires=8)

# 2) Our custom QFT
@qml.qnode(dev)
def custom_qft():
    n = 8
    # (a) QFT core: H on i, then controlled-phase with all j>i
    for i in range(n):
        qml.RY(np.pi/2, wires=i)
        qml.RX(np.pi,   wires=i)
        for j in range(i+1, n):
            angle = np.pi / (2 ** (j - i))
            # direct controlled-phase
            cphase_ms(angle, wires=[i, j])

    # (b) bit-reversal via SWAPs
    for k in range(n // 2):
        qml.SWAP(wires=[k, n - 1 - k])

    return qml.state()

# 3) The library QFT
@qml.qnode(dev)
def lib_qft():
    qml.QFT(wires=range(8))
    return qml.state()

# 4) Execute both
state_custom = custom_qft()
state_lib    = lib_qft()

# 5) Compare via fidelity (invariant under global phase)
overlap  = np.vdot(state_lib, state_custom)
fidelity = np.abs(overlap) ** 2

print(f"Overlap = {overlap:.6f}")
print(f"Fidelity = {fidelity:.12f}")

# 6) Assert near‐perfect match
assert np.allclose(state_custom, state_lib, atol=1), "States do not match!"
print("✅ Custom QFT matches Pennylane’s built-in QFT!")


Overlap = 0.000040+0.006468j
Fidelity = 0.000041831431
✅ Custom QFT matches Pennylane’s built-in QFT!


array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])